# Tähelepanumehhanismid ja transformerid

Üks peamisi korduvvõrkude puudusi on see, et kõik sõnad järjestuses mõjutavad tulemust võrdselt. See põhjustab standardsete LSTM kodeerija-dekodeerija mudelite puhul alamoptimaalset jõudlust järjestuste vaheliste ülesannete, nagu nimede tuvastamine ja masintõlge, lahendamisel. Tegelikkuses avaldavad teatud sisendjärjestuse sõnad sageli suuremat mõju väljundjärjestusele kui teised.

Vaatame järjestuste vahelise mudeli, näiteks masintõlke, toimimist. Seda rakendatakse kahe korduvvõrgu abil, kus üks võrk (**kodeerija**) koondab sisendjärjestuse varjatud olekusse ja teine, **dekodeerija**, laotab selle varjatud oleku tõlgitud tulemuseks. Selle lähenemise probleem seisneb selles, et võrgu lõplik olek ei suuda hästi meeles pidada lause algust, mis põhjustab mudeli kehva kvaliteeti pikkade lausete puhul.

**Tähelepanumehhanismid** pakuvad võimalust kaaluda iga sisendvektori kontekstuaalset mõju RNN-i iga väljundprognoosi puhul. Seda rakendatakse, luues otseteid sisend-RNN-i vaheolekute ja väljund-RNN-i vahel. Sel viisil, kui genereeritakse väljundisümbolit $y_t$, arvestame kõiki sisendi varjatud olekuid $h_i$, erinevate kaalukoefitsientidega $\alpha_{t,i}$.

![Pilt, mis näitab kodeerija/dekodeerija mudelit koos aditiivse tähelepanukihiga](../../../../../translated_images/et/encoder-decoder-attention.7a726296894fb567.webp)
*Kodeerija-dekodeerija mudel koos aditiivse tähelepanumehhanismiga [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), viidatud [sellest blogipostitusest](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Tähelepanumaatriks $\{\alpha_{i,j}\}$ esindab, millisel määral teatud sisendsõnad mõjutavad antud sõna genereerimist väljundjärjestuses. Allpool on näide sellisest maatriksist:

![Pilt, mis näitab näidisalineerimist, mille leidis RNNsearch-50, võetud Bahdanau - arviz.org](../../../../../translated_images/et/bahdanau-fig3.09ba2d37f202a6af.webp)

*Joonis võetud [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Joonis 3)*

Tähelepanumehhanismid vastutavad suure osa praeguse või peaaegu praeguse tipptaseme eest loomuliku keele töötlemises. Tähelepanu lisamine suurendab aga oluliselt mudeli parameetrite arvu, mis põhjustas RNN-ide skaleerimisprobleeme. RNN-ide skaleerimise peamine piirang on see, et mudelite korduv olemus muudab treeningu rühmitamise ja paralleelimise keeruliseks. RNN-is tuleb järjestuse iga element töödelda järjestikuses järjekorras, mis tähendab, et seda ei saa lihtsalt paralleelselt töödelda.

Tähelepanumehhanismide kasutuselevõtt koos selle piiranguga viis tänapäeval tuntud ja kasutatavate tipptasemel transformerimudelite loomiseni, nagu BERT ja OpenGPT3.

## Transformerimudelid

Selle asemel, et edastada iga eelneva prognoosi konteksti järgmisesse hindamissammu, kasutavad **transformerimudelid** **positsioonikodeeringuid** ja tähelepanu, et haarata antud sisendi konteksti etteantud tekstivahemikus. Allolev pilt näitab, kuidas positsioonikodeeringud koos tähelepanuga suudavad konteksti haarata antud vahemikus.

![Animeeritud GIF, mis näitab, kuidas hindamisi tehakse transformerimudelites.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Kuna iga sisendpositsioon kaardistatakse sõltumatult iga väljundpositsiooniga, suudavad transformerid paremini paralleelselt töötada kui RNN-id, mis võimaldab palju suuremaid ja väljendusrikkamaid keelemudeleid. Iga tähelepanukeskus saab kasutada erinevate sõnadevaheliste seoste õppimiseks, mis parandab loomuliku keele töötlemise ülesandeid.

**BERT** (Bidirectional Encoder Representations from Transformers) on väga suur mitmekihiline transformerivõrk, millel on 12 kihti *BERT-base* jaoks ja 24 kihti *BERT-large* jaoks. Mudel treenitakse esmalt suure tekstikorpuse (Wikipedia + raamatud) peal, kasutades juhendamata treeningut (maskeeritud sõnade ennustamine lauses). Treeningu käigus omandab mudel märkimisväärse taseme keele mõistmist, mida saab seejärel kasutada teiste andmekogumite peal peenhäälestamise abil. Seda protsessi nimetatakse **ülekandeõppeks**.

![Pilt aadressilt http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/et/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Transformeriarhitektuuridel, nagu BERT, DistilBERT, BigBird, OpenGPT3 ja teised, on palju variatsioone, mida saab peenhäälestada. [HuggingFace pakett](https://github.com/huggingface/) pakub repositooriumi paljude nende arhitektuuride treenimiseks PyTorchiga.

## BERT-i kasutamine tekstiklassifikatsiooniks

Vaatame, kuidas saame kasutada eelnevalt treenitud BERT-mudelit meie traditsioonilise ülesande lahendamiseks: järjestuste klassifikatsioon. Klassifitseerime oma algse AG News andmekogumi.

Kõigepealt laadime HuggingFace'i teegi ja meie andmekogumi:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Kuna me kasutame eeltreenitud BERT-mudelit, peame kasutama sellele vastavat tokeniseerijat. Kõigepealt laadime tokeniseerija, mis on seotud eeltreenitud BERT-mudeliga.

HuggingFace'i teegis on eeltreenitud mudelite repository, mida saab kasutada lihtsalt nende nimede määramisega `from_pretrained` funktsioonide argumentidena. Kõik vajalikud binaarfailid mudeli jaoks laaditakse automaatselt alla.

Siiski, teatud juhtudel võib olla vaja laadida oma mudelid. Sel juhul saab määrata kataloogi, mis sisaldab kõiki asjakohaseid faile, sealhulgas tokeniseerija parameetreid, `config.json` faili mudeli parameetritega, binaarkaalu jms.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` objekt sisaldab `encode` funktsiooni, mida saab otse kasutada teksti kodeerimiseks:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Seejärel loome iteratorid, mida kasutame treeningu ajal andmetele juurdepääsuks. Kuna BERT kasutab oma kodeerimisfunktsiooni, peame määratlema täitmisfunktsiooni, mis on sarnane varem määratletud `padify` funktsiooniga:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Meie puhul kasutame eelnevalt treenitud BERT-mudelit nimega `bert-base-uncased`. Laadime mudeli `BertForSequenceClassification` paketiga. See tagab, et meie mudelil on juba vajalik klassifikatsiooni arhitektuur, sealhulgas lõplik klassifikaator. Näete hoiatust, mis ütleb, et lõpliku klassifikaatori kaalud ei ole algväärtustatud ja mudel vajab eeltreenimist - see on täiesti normaalne, sest just seda me kavatseme teha!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Nüüd oleme valmis treenimist alustama! Kuna BERT on juba eelnevalt treenitud, tahame alustada üsna väikese õppemääraga, et mitte algseid kaalusid rikkuda.

Kogu raske töö teeb mudel `BertForSequenceClassification`. Kui kutsume mudeli treeningandmetele, tagastab see nii kaotuse kui ka võrgu väljundi sisendminiparti jaoks. Kasutame kaotust parameetrite optimeerimiseks (`loss.backward()` teeb tagasipassi) ja `out` treeningtäpsuse arvutamiseks, võrreldes saadud silte `labs` (arvutatud `argmax` abil) oodatud `labels`-iga.

Protsessi kontrollimiseks kogume kaotuse ja täpsuse mitme iteratsiooni jooksul ning prindime need iga `report_freq` treeningtsükli järel.

See treening võtab tõenäoliselt üsna kaua aega, seega piirame iteratsioonide arvu.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Sa võid näha (eriti kui suurendad iteratsioonide arvu ja ootad piisavalt kaua), et BERT-i klassifikatsioon annab meile üsna hea täpsuse! See on tingitud sellest, et BERT mõistab juba keele struktuuri üsna hästi ning meil on vaja ainult lõplikku klassifikaatorit peenhäälestada. Kuid kuna BERT on suur mudel, võtab kogu treeningprotsess palju aega ja nõuab tõsist arvutusvõimsust! (GPU ja eelistatavalt rohkem kui üks).

> **Note:** Meie näites oleme kasutanud ühte väikseimat eelnevalt treenitud BERT-i mudelit. On olemas suuremaid mudeleid, mis tõenäoliselt annavad paremaid tulemusi.


## Mudeli jõudluse hindamine

Nüüd saame hinnata oma mudeli jõudlust testandmestiku põhjal. Hindamistsükkel on treeningtsükliga üsna sarnane, kuid me ei tohiks unustada mudeli hindamisrežiimile lülitamist, kutsudes välja `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Peamine mõte

Selles üksuses nägime, kui lihtne on võtta **transformers** teegist eelnevalt treenitud keelemudel ja kohandada see meie tekstiklassifitseerimise ülesande jaoks. Samamoodi saab BERT-mudeleid kasutada üksuste tuvastamiseks, küsimustele vastamiseks ja muude NLP ülesannete jaoks.

Transformer-mudelid esindavad praegust tipptaset NLP-s ning enamasti peaksid need olema esimene lahendus, millega alustad, kui rakendad kohandatud NLP lahendusi. Siiski on äärmiselt oluline mõista korduvate närvivõrkude põhiprintsiipe, mida selles moodulis käsitleti, kui soovid luua edasijõudnud närvivõrke.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
